### Задание №1
Скачайте текст "Литературных анекдотов". Напишите функцию, которая будет читать файл, лемматизировать текст с помощью pymystem3 и записывать результат в новый файл. У функции должно бы два аргумента: путь к исходному файлу и путь к файлу с лемматизированным текстом. Вызов функции тоже должен быть прописан в решении.


In [31]:
import os, re, json
from pymystem3 import Mystem
from collections import Counter


In [18]:
# Создаем функцию
def lemmas_to_file(file_in, file_out):
    
    # Читаем файл с анекдотами
    with open(file_in, 'r', encoding='utf-8') as f:
        text = f.read()

    #Лемматизируем
    m = Mystem()
    lemmas = m.lemmatize(text)
    text_lemma = ''.join(lemmas) # соединяем в строку

    # записываем текст в файл
    with open(file_out, 'w', encoding='utf-8') as f:
        f.write(text_lemma)


In [21]:
lemmas_to_file(input("Введите путь к исходному файлу\n"), input("Задайте имя конечного файла\n"))

Введите путь к исходному файлу
literary_anecdotes.txt
Задайте имя конечного файла
lemmatized.txt


### Задание №2
Очистите лемматизированный текст от стоп-слов и посчитайте ipm для оставшихся. Выведите 20 самых частотных по этому параметру слов.


In [30]:
# Очищаем от пунктуации
text_halfclean = [w.strip("!\"#$%&'()*+,./:;<=>?@[\]^_`{|}~„“«»†*—/\-—") for w in text_lemma.split()] 
text_clean = [w for w in text_halfclean if len(w) != 0] # очищаем от пустых элементов



# Очищаем от стоп слов

# Открываем файл со стоп-словами
with open('rus_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read()

stopwords = stopwords.split('\n') # делаем список по переносу строки
text_clean = [w for w in text_clean if w not in stopwords] # в новый список все слова, кроме стоп слов


# Считаем и выводим
counts = Counter(text_clean)
print(counts.most_common(20))

[('пушкин', 54), ('толстой', 33), ('гоголь', 31), ('однажды', 29), ('лев', 25), ('любить', 20), ('достоевский', 19), ('тургенев', 15), ('ребенок', 14), ('царствие', 13), ('окно', 12), ('тверской', 12), ('бульвар', 12), ('идти', 12), ('лермонтов', 12), ('приходить', 11), ('федор', 11), ('михайлович', 11), ('герцен', 10), ('небесный', 10)]


### Задание №3
Сделайте полный морфологический разбор исходного текста. Напишите регулярное выражение, которое будет извлекать из тега только часть речи. Пройдитесь циклом по списку с разборами, который выдал pymystem3, извлекая из каждого разбора форму слова и его часть речи и записывая их в новый словарь (форма -- ключ, часть речи -- значение). Посчитайте абсолютную частоту для всех частей речи, а затем относительнную частоту (абсолютная частота / длина текста в словах).


In [53]:
# Открываем файл с исходным текстом
with open('literary_anecdotes.txt', 'r', encoding='utf-8') as f:
    story = f.read()

# Анализируем
story_analyzed = m.analyze(story)
 
# создаём словарь с только грамматикой
words = {}
for word in story_analyzed:
    try:
        form = word['text']
        grammar = word['analysis'][0]['gr']
        words[form] = grammar
    except KeyError:
        pass
    except IndexError:
        pass

# Компилируем регулярку для вытаскивания части речи
pos = re.compile('[A-Z]+')

# Делаем пустой словарь
words_pos = {}

# Добавляем в словарь ключ из предыдущего словаря и значение которое даёт регулярка
for k, v in words.items():
    words_pos[k] = pos.match(v).group(0)

In [54]:
len(words_pos)

1355

In [60]:
# Считаем абсолютную частоту частей речи.
# Надо создать новый словарь где ключом будет название части речи, а значением — его абс. частотность

pos_abs_freq = {}
for k, v in words_pos.items():
    pos_abs_freq[v] = sum(value == v for value in words_pos.values()) # делаем ключом значение из пред. словаря, а значением — количество (мы это, конечно не проходили, я взяла из интернета)
    
pos_abs_freq

{'PR': 38,
 'S': 507,
 'V': 427,
 'A': 98,
 'ADV': 88,
 'SPRO': 55,
 'CONJ': 20,
 'PART': 29,
 'ADVPRO': 24,
 'APRO': 48,
 'ANUM': 4,
 'INTJ': 8,
 'NUM': 9}

In [59]:
# Считаем относительную частоту частей речи.


# Надо создать новый словарь где ключом будет название части речи, а значением — его относительная частотность 
# (т.е. делим абсолютную на длину текста в словах)
pos_rel_freq = {}
for k, v in pos_abs_freq.items():
    pos_rel_freq[k] = (sum(value == v for value in pos_abs_freq.values()))/len(story.split())
pos_rel_freq

{'PR': 0.00034211426616489907,
 'S': 0.00034211426616489907,
 'V': 0.00034211426616489907,
 'A': 0.00034211426616489907,
 'ADV': 0.00034211426616489907,
 'SPRO': 0.00034211426616489907,
 'CONJ': 0.00034211426616489907,
 'PART': 0.00034211426616489907,
 'ADVPRO': 0.00034211426616489907,
 'APRO': 0.00034211426616489907,
 'ANUM': 0.00034211426616489907,
 'INTJ': 0.00034211426616489907,
 'NUM': 0.00034211426616489907}